# TP3 : Équations différentielles
## PHY-3500: Physique numérique (H2021) - 19 mars 2021


Mélissa Cinq-Mars (111 179 946)

Pierre-Olivier Janvier (111 187 987)

Maxence Larose (111 188 447)

In [11]:
#import packages
import astropy.constants as const
import numpy as np

### Mécaniques céleste - comètes

#### Question a)

L'équation différentielle qui régit le mouvement du système comète-soleil est donné par:
$$ m \dfrac{d^2\textbf{r}}{dt^2} = - \left (\dfrac{GMm}{r^2} \right ) \dfrac{\textbf{r}}{r} $$

Puisqu'on considère que le mouvement est confiné dans un plan, les deux dimensions nécessaires pour décrire le mouvement sont données par les équations suivantes:
$$ \dfrac{d^2x}{dt^2} = -GM \dfrac{x}{r^3} $$


$$ \dfrac{d^2 y}{dt^2} = -GM \dfrac{y}{r^3} $$

Pour simplifier la démarche analytique, il est préférable de réécrire ces deux équations différentielles du deuxième ordre en quatre équations différentielles du premier ordre en utilisant un changement de variable suivant:

$$ \dfrac{dx}{dt} = v_x   \hspace{2cm}  \dfrac{dv_x}{dt} = -GM \dfrac{x}{r^3} $$

$$ \dfrac{dy}{dt} = v_y   \hspace{2cm}   \dfrac{dv_y}{dt} = -GM \dfrac{y}{r^3} $$

#### Question b)

In [30]:
G = const.G
M = const.M_sun

#mixte en exemples 8.5 et 8.6

def f(r,t):
    G = const.G
    M = const.M_sun
    
    x = r[0]
    y = r[1]
    z = (x**2 + y**2)**0.5
    
    vx = r[2]
    vy = r[3]
    
    f_x = vx
    f_y = vy
    f_vx = -G*M*x*z**-3
    f_vy = -G*M*y*z**-3
    
    return np.array([f_vx, f_vy], float)

x0 = 4*10**9
y0 = 0

v_x = 0
v_y = 500

a = 0
b = 100
N = 100
h = (b-a)/N

tpoints = np.arange(a, b, h)
xpoints = []
ypoints = []

r = np.array([v_x, v_y], float)
for tx in tpoints:
    G = const.G
    M = const.M_sun
    v_x = 0
    v_y = 500
    
    xpoints.append(x)
    
    k1 = h*f(r, t)
    k2 = h*f(r+0.5*k1, t+0.5*h)
    k3 = h*f(r+0.5*k2, t+0.5*h)
    k4 = h*f(r+k3, t+h)
    
    r += (k1 + 2*k2 + 2*k3 + k4)/6


IndexError: index 2 is out of bounds for axis 0 with size 2

#### Question c)

### Mécanique céleste - la Terre

#### Question d)

#### Question e)

#### Question f)

#### Question g)

### Mécanique céleste - Mars

#### Question h)